In [1]:
import torch.nn   as nn
import GM_library as GML
import numpy      as np
import torch

from os                   import path, listdir
from torch_geometric.data import Data

import sys
sys.path.append('../')
import MP.MP_library as MPL

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Number of graphs to predict
N_predictions = 10

# Number of diffusing and denoising steps
n_t_steps = 2

# Decay of parameter alpha
noise_contribution = 0.15
alpha_decay = 0.5 * (1 - noise_contribution**2)

# Dropouts for node and edge models (independent of each other)
dropout_node = 0.2
dropout_edge = 0.2

# Define box shape
L = [20, 20, 20]

# Target to generate new crystals
target = 'GM_EPA'

# In case database is created from scratch (otherwise, it is not being used)
DB_path = '../MP/Loaded_EMP'

input_folder    = 'models'
target_folder   = f'{input_folder}/{target}'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

In [3]:
# Number of graphs to predict
N_predictions = 10

# Number of diffusing and denoising steps, which can be different
n_denoising_steps = 2

# Dropouts for node and edge models (independent of each other)
dropout_node = 0.2
dropout_edge = 0.2

# Define box shape
L = [20, 20, 20]

# Target to generate new crystals
target = 'GM_EPA'

# In case database is created from scratch (otherwise, it is not being used)
DB_path = '../MP/Loaded_EMP'

input_folder    = 'models'
target_folder   = f'{input_folder}/{target}'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [4]:
labels_name         = f'{target_folder}/labels.pt'
dataset_name        = f'{target_folder}/dataset.pt'
dataset_name_std    = f'{target_folder}/standardized_dataset.pt'
parameters_name_std = f'{target_folder}/standardized_parameters.pt'  # Parameters for rescaling the predictions

# Load the standardized dataset, with corresponding labels and parameters
dataset    = torch.load(dataset_name_std)
parameters = torch.load(parameters_name_std)

# Assigning parameters accordingly
target_mean, feat_mean, edge_mean, target_std, edge_std, feat_std, scale = parameters

# Defining target factor
target_factor = target_std / scale

In [5]:
# Calculate the mean and standard deviation of the number of nodes
total_nodes = torch.tensor([data.num_nodes for data in dataset])
mean_nodes  = torch.mean(total_nodes.float()).item()
std_nodes   = torch.std(total_nodes.float()).item()

mean_nodes, std_nodes

(620.1802978515625, 163.6297149658203)

# Loading the model

In [6]:
# Determine number of features in dataset, considering the t_step information
n_features = dataset[0].num_node_features + 1

# Instantiate the models for nodes and edges
node_model = GML.nGCNN(n_features, dropout_node).to(device)
node_model.load_state_dict(torch.load(node_model_name))
node_model.eval()

edge_model = GML.eGCNN(n_features, dropout_edge).to(device)
edge_model.load_state_dict(torch.load(edge_model_name))
edge_model.eval()

print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)


Node GCNN:
nGCNN(
  (conv1): GraphConv(5, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 5)
)

Edge GCNN:
eGCNN(
  (linear1): Linear(in_features=6, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)


# Generating new cystals

In [7]:
# Predicting loop
predicted_dataset = []
with torch.no_grad():
    for i in range(N_predictions):
        # Get random number of nodes
        n_nodes = int(np.random.normal(mean_nodes, std_nodes))
        
        # Diffuse the graph with some noise
        diffused_graph = GML.get_random_graph(n_nodes, n_features-1)  # CHECK THIS FUNCTION
        
        # Denoise the diffused graph
        print(f'Denoising...')
        denoised_graph, _ = GML.denoise(diffused_graph, n_t_steps, node_model, edge_model,
                                        s=alpha_decay)
        
        # Append generated graph
        predicted_dataset.append(denoised_graph)

# Revert stardadization
denoised_graph = GML.revert_standardize_dataset(predicted_dataset, parameters)

Denoising...
Data(x=[449, 4], edge_index=[2, 100576], edge_attr=[100576])
Denoising...
Data(x=[593, 4], edge_index=[2, 175528], edge_attr=[175528])
Denoising...
Data(x=[799, 4], edge_index=[2, 318801], edge_attr=[318801])
Denoising...
Data(x=[335, 4], edge_index=[2, 55945], edge_attr=[55945])
Denoising...
Data(x=[342, 4], edge_index=[2, 58311], edge_attr=[58311])
Denoising...
Data(x=[867, 4], edge_index=[2, 375411], edge_attr=[375411])
Denoising...
Data(x=[944, 4], edge_index=[2, 445096], edge_attr=[445096])
Denoising...
Data(x=[758, 4], edge_index=[2, 286903], edge_attr=[286903])
Denoising...
Data(x=[430, 4], edge_index=[2, 92235], edge_attr=[92235])
Denoising...
Data(x=[492, 4], edge_index=[2, 120786], edge_attr=[120786])


In [9]:
graph_raw      = predicted_dataset[0]
graph_discrete = GML.discretize_graph(graph_raw)

In [10]:
node_loss, edge_loss = GML.get_graph_losses(graph_raw, graph_discrete)
node_loss, edge_loss

(tensor(30.7057), tensor(0.))

In [68]:
aa = GML.revert_standardize_dataset(dataset, parameters)

In [69]:
direct_positions = GML.graph_to_direct_positions(aa[0], L)

In [37]:
import importlib
importlib.reload(GML)

<module 'GM_library' from '/Users/cibran/Work/UPC/GenerativeModels/GM_library.py'>

In [26]:
denoised_graph = GML.revert_standardize_dataset([dataset[0]], parameters)[0]

In [28]:
GML.POSCAR_graph_encoding(denoised_graph, L, POSCAR_name=None, POSCAR_directory='./')

<_io.TextIOWrapper name='.//POSCAR' mode='w' encoding='UTF-8'>